In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

In [3]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [4]:
database = "sqlite_db_pythonsqlite.db"

# create a database connection
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks(conn)

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [5]:
#Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members
def select_all_tasks1(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
    SELECT f2.name, s2.Revenue
    FROM Facilities as f2
    LEFT JOIN
        (SELECT f1.facid, f1.name, SUM(s.total_cost) as Revenue
        FROM Facilities as f1
        LEFT JOIN
            (SELECT b.bookid, f.facid, m.memid,f.guestcost, f.membercost, b.slots,
            CASE WHEN m.memid = 0 THEN (f.guestcost * b.slots) 
            ELSE (f.membercost * b.slots) END AS total_cost
            FROM Bookings AS b
            INNER JOIN Facilities AS f ON b.facid = f.facid
            INNER JOIN Members AS m ON b.memid = m.memid) as s
        ON f1.facid = s.facid
        GROUP BY f1.name) as s2
    ON f2.facid = s2.facid
    WHERE s2.Revenue < 1000
    ORDER BY Revenue DESC
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [6]:
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks1(conn)

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


In [7]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
def select_all_tasks11(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
    SELECT m1.surname AS Member_Last_Name, m1.firstname AS Member_First_Name, 
    CASE WHEN m2.memid = 0 THEN ''
    ELSE m2.surname END as Recommended_By_Surname, 
    CASE WHEN m2.memid = 0 THEN ''
    ELSE m2.firstname END as Recommended_By_Firstname
    FROM Members as m1
    LEFT JOIN Members as m2
    ON m1.recommendedby = m2.memid
    WHERE m1.memid != 0
    ORDER BY Member_Last_Name
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
    
    df = pd.DataFrame(rows)
    df.columns = ['Member_Last_Name','Member_First_Name','Recommended_By_Last_Name','Recommended_By_First_Name']

In [8]:
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks11(conn)

2.6.0
2. Query all tasks
('Bader', 'Florence', 'Stibbons', 'Ponder')
('Baker', 'Anne', 'Stibbons', 'Ponder')
('Baker', 'Timothy', 'Farrell', 'Jemima')
('Boothe', 'Tim', 'Rownam', 'Tim')
('Butters', 'Gerald', 'Smith', 'Darren')
('Coplin', 'Joan', 'Baker', 'Timothy')
('Crumpet', 'Erica', 'Smith', 'Tracy')
('Dare', 'Nancy', 'Joplette', 'Janice')
('Farrell', 'Jemima', None, None)
('Farrell', 'David', None, None)
('Genting', 'Matthew', 'Butters', 'Gerald')
('Hunt', 'John', 'Purview', 'Millicent')
('Jones', 'David', 'Joplette', 'Janice')
('Jones', 'Douglas', 'Jones', 'David')
('Joplette', 'Janice', 'Smith', 'Darren')
('Mackenzie', 'Anna', 'Smith', 'Darren')
('Owen', 'Charles', 'Smith', 'Darren')
('Pinker', 'David', 'Farrell', 'Jemima')
('Purview', 'Millicent', 'Smith', 'Tracy')
('Rownam', 'Tim', None, None)
('Rumney', 'Henrietta', 'Genting', 'Matthew')
('Sarwin', 'Ramnaresh', 'Bader', 'Florence')
('Smith', 'Darren', None, None)
('Smith', 'Tracy', None, None)
('Smith', 'Jack', 'Smith', 'Darre

In [28]:
#Q12: Find the facilities with their usage by member, but not guests 
def select_all_tasks112(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT f.name as Facility, (m. firstname || ' ' || m.surname) as Member_name, COUNT(f.facid) as Usage
FROM Facilities as f
INNER JOIN Members as m
WHERE m.memid !=0
GROUP BY Facility, Member_name
      """

    cur.execute(query1)
 
    rows = cur.fetchall()
    
    df = pd.DataFrame(rows)
    df.columns = ['Facility','Member_Name','Usage']
    
    print(df)


In [29]:
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks112(conn)

2.6.0
2. Query all tasks
            Facility       Member_Name  Usage
0    Badminton Court    Anna Mackenzie      1
1    Badminton Court        Anne Baker      1
2    Badminton Court      Burton Tracy      1
3    Badminton Court      Charles Owen      1
4    Badminton Court      Darren Smith      2
..               ...               ...    ...
256   Tennis Court 2  Ramnaresh Sarwin      1
257   Tennis Court 2        Tim Boothe      1
258   Tennis Court 2        Tim Rownam      1
259   Tennis Court 2     Timothy Baker      1
260   Tennis Court 2       Tracy Smith      1

[261 rows x 3 columns]


In [43]:
#Q13: Find the facilities usage by month, but not guests
def select_all_tasks113(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT f.name as Facility, s.bookmonth as Month, COUNT(s.bookid)
FROM Facilities as f
INNER JOIN 
    (SELECT b.bookid, strftime('%m', b.starttime) as bookmonth, b.facid
    FROM Bookings as b
    INNER JOIN Members as m
    WHERE m.memid != 0) as s
ON f.facid = s.facid
GROUP BY Facility, Month

      """

    cur.execute(query1)
 
    rows = cur.fetchall()
    
    df = pd.DataFrame(rows)
    df.columns = ['Facility','Month','Usage']
    
    print(df)


In [44]:
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks113(conn)

2.6.0
2. Query all tasks
           Facility Month  Usage
0   Badminton Court    07   1680
1   Badminton Court    08   4380
2   Badminton Court    09   5430
3    Massage Room 1    07   3690
4    Massage Room 1    08   6720
5    Massage Room 1    09   8460
6    Massage Room 2    07    360
7    Massage Room 2    08   1200
8    Massage Room 2    09   1770
9        Pool Table    07   3300
10       Pool Table    08   8730
11       Pool Table    09  13050
12    Snooker Table    07   2250
13    Snooker Table    08   4770
14    Snooker Table    09   6300
15     Squash Court    07   2250
16     Squash Court    08   5100
17     Squash Court    09   5850
18     Table Tennis    07   1530
19     Table Tennis    08   4410
20     Table Tennis    09   6150
21   Tennis Court 1    07   2640
22   Tennis Court 1    08   4380
23   Tennis Court 1    09   5220
24   Tennis Court 2    07   2040
25   Tennis Court 2    08   4470
26   Tennis Court 2    09   5160
